# 🎯 E-commerce Customer Churn - Industry-Grade EDA

**Dataset:** 5,630 customers | 20 features  
**Business Goal:** Identify churn risk factors that are **knowable before churn** (production-safe)  
**Critical Constraint:** Avoid data leakage (features observed after bad experience)

---

## 📋 EDA Objectives

This EDA will answer:
1. What % of customers churn? (Class imbalance assessment)
2. Which features are **safe** vs **risky** for production?
3. What business hypotheses can we test?
4. Which features need special handling (lagging, exclusion)?

**NOT just plotting** - we're making **modeling decisions**.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 📥 Step 1: Load Data & Initial Inspection

In [ ]:
df = pd.read_csv('../data/raw/ecommerce_churn.csv')
print(f"Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")
df.head()

## 🔍 Step 2: Data Quality Assessment

**Why this matters:**  
- Wrong dtypes → model errors
- Missing values → imputation strategy needed
- Duplicates → data integrity issues

In [ ]:
# Data types
print("="*80)
print("DATA TYPES")
print("="*80)
print(df.dtypes)

# Missing values
print("\n" + "="*80)
print("MISSING VALUES ANALYSIS")
print("="*80)
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing_Count': missing, 'Missing_%': missing_pct})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
    print(f"\n⚠️ {len(missing_df)} columns have missing values - imputation strategy required")
else:
    print("✓ No missing values found")

# Duplicates
if 'CustomerID' in df.columns:
    duplicates = df['CustomerID'].duplicated().sum()
    print(f"\nDuplicate CustomerIDs: {duplicates}")

## 🎯 Step 3: Target Variable Analysis (Churn)

**Business Question:** What is the baseline churn rate?  
**Why this matters:** Determines if we need class balancing (SMOTE/ADASYN)

**Decision Criteria:**
- Minority class < 10% → SEVERE imbalance → SMOTE CRITICAL
- Minority class 10-30% → MODERATE imbalance → Resampling recommended
- Minority class > 30% → Balanced → Direct modeling possible

In [ ]:
# Churn distribution
churn_rate = df['Churn'].mean() * 100
churned = df['Churn'].sum()
retained = len(df) - churned

print("="*80)
print("CHURN ANALYSIS")
print("="*80)
print(f"\nChurn Rate: {churn_rate:.2f}%")
print(f"  - Churned: {churned:,} customers")
print(f"  - Retained: {retained:,} customers")

# Class imbalance assessment
minority_pct = min(churn_rate, 100 - churn_rate)
print(f"\nMinority Class: {minority_pct:.2f}%")

if minority_pct < 10:
    print("\n🚨 DECISION: SEVERE IMBALANCE → SMOTE/ADASYN is CRITICAL")
elif minority_pct < 30:
    print("\n⚠️ DECISION: MODERATE IMBALANCE → Class resampling recommended")
else:
    print("\n✓ DECISION: Balanced dataset → Direct modeling possible")

# Visualization
plt.figure(figsize=(8, 5))
df['Churn'].value_counts().plot(kind='bar', color=['green', 'red'], edgecolor='black')
plt.title('Churn Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Churn (0=Retained, 1=Churned)')
plt.ylabel('Customer Count')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
for i, v in enumerate(df['Churn'].value_counts()):
    plt.text(i, v + 50, str(v), ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

## 🟢🟡🔴 Step 4: Feature Trust Levels (DATA LEAKAGE ASSESSMENT)

**THIS IS THE MOST CRITICAL PART OF EDA.**

We categorize features by **production safety**:

### 🟢 SAFE Features (Low Leakage Risk)
- Describe customer **STATE** (not behavior after dissatisfaction)
- Available **before** churn decision
- Safe for production deployment

### 🟡 QUESTIONABLE Features (Possible Leakage)
- **May be observed AFTER bad experience**
- Could act as **post-churn signals**
- Example: Satisfaction score drops AFTER poor service
- **Decision:** Use with caution, document leakage risk

### 🔴 HIGH-RISK Features (Temporal/Behavioral)
- "Last interaction" variables
- Strongly correlated with churn (potential leakage)
- **Decision:** If correlation > 0.9 → likely leakage, investigate deeply

**Why this matters:**  
Using leakage features gives **fake 99% accuracy** in training but **fails in production** because those features aren't available at prediction time.

**Production Reality:**  
When predicting churn for a customer, we can only use features **knowable at that moment**, not features that emerge during/after the churn process.

In [ ]:
# Feature categorization
safe_features = [
    'Tenure', 'CityTier', 'WarehouseToHome', 'NumberOfDeviceRegistered',
    'NumberOfAddress', 'PreferredLoginDevice', 'PreferredPaymentMode',
    'Gender', 'PreferedOrderCat', 'MaritalStatus'
]

questionable_features = [
    'SatisfactionScore',  # May drop AFTER bad experience
    'Complain',           # Happens AFTER dissatisfaction
    'CashbackAmount',     # Could be result of retention efforts
    'HourSpendOnApp'      # May decrease DURING churn process
]

high_risk_features = [
    'DaySinceLastOrder',              # Temporal - increases as customer disengages
    'OrderCount',                      # Behavioral - drops during churn
    'OrderAmountHikeFromlastYear',    # Could be post-churn signal
    'CouponUsed'                       # May be retention attempt
]

print("="*80)
print("FEATURE TRUST LEVELS (LEAKAGE RISK ASSESSMENT)")
print("="*80)

print("\n🟢 SAFE FEATURES (Production-ready):")
for feat in safe_features:
    if feat in df.columns:
        print(f"   ✓ {feat}")
print("   → Customer STATE variables")
print("   → DECISION: Safe for modeling")

print("\n🟡 QUESTIONABLE FEATURES (Use with caution):")
for feat in questionable_features:
    if feat in df.columns:
        print(f"   ⚠️ {feat}")
print("   → May be observed AFTER bad experience")
print("   → DECISION: Use but DOCUMENT leakage risk in production")

print("\n🔴 HIGH-RISK FEATURES (Temporal/behavioral):")
for feat in high_risk_features:
    if feat in df.columns:
        print(f"   🚨 {feat}")
print("   → 'Last interaction' variables")
print("   → DECISION: Check correlation; if > 0.9, likely leakage")

## 📊 Step 5: Correlation Analysis (With Leakage Check)

**Hypothesis:** Features with |correlation| > 0.5 are strong churn indicators  
**Leakage Check:** If correlation > 0.9, feature may be post-churn signal

**CRITICAL:** Correlation ≠ Causation  
- High correlation means feature is a **strong indicator**
- Does NOT mean feature **causes** churn
- Could be a **proxy** for underlying dissatisfaction

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
correlations = df[numerical_cols].corrwith(df['Churn']).sort_values(ascending=False)

print("="*80)
print("CORRELATION WITH CHURN")
print("="*80)
print(correlations)

# Flag high correlations
print("\n" + "="*80)
print("LEAKAGE RISK ASSESSMENT")
print("="*80)

high_corr = correlations[abs(correlations) > 0.5]
if len(high_corr) > 0:
    print("\n🚨 HIGH CORRELATION (|r| > 0.5):")
    for feat, corr in high_corr.items():
        if feat != 'Churn':
            print(f"   {feat}: {corr:.3f}")
    print("\n⚠️ CRITICAL QUESTION: Are these features known BEFORE churn?")
    print("   If NO → Data leakage risk!")
else:
    print("\n✓ No extremely high correlations (|r| > 0.5)")

# Moderate correlations
moderate_corr = correlations[(abs(correlations) > 0.2) & (abs(correlations) <= 0.5)]
if len(moderate_corr) > 0:
    print("\n📊 MODERATE CORRELATION (0.2 < |r| <= 0.5):")
    for feat, corr in moderate_corr.items():
        if feat != 'Churn':
            print(f"   {feat}: {corr:.3f}")
    print("   → These are strong indicators, likely safe")

# Visualization
plt.figure(figsize=(10, 8))
correlations_plot = correlations.drop('Churn')
colors = ['red' if x > 0 else 'green' for x in correlations_plot]
correlations_plot.plot(kind='barh', color=colors, edgecolor='black')
plt.title('Correlation with Churn (NOT Causation!)', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.axvline(x=0.5, color='red', linestyle=':', alpha=0.5, label='High correlation threshold')
plt.axvline(x=-0.5, color='red', linestyle=':', alpha=0.5)
plt.legend()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 📅 Step 6: Tenure Analysis

**Business Hypothesis:** Customers with lower tenure are more likely to churn  
**Reasoning:** New customers haven't built loyalty; easier to switch  
**Expected:** Strong negative correlation between Tenure and Churn

**Decision Criteria:**
- If correlation < -0.2 → Hypothesis CONFIRMED → Use in model
- If correlation ≈ 0 → Hypothesis REJECTED → Investigate why

In [ ]:
print("="*80)
print("TENURE ANALYSIS")
print("="*80)

# Statistics by churn status
print("\nTenure Statistics by Churn Status:")
print(df.groupby('Churn')['Tenure'].describe())

# Correlation
tenure_corr = df['Tenure'].corr(df['Churn'])
print(f"\nTenure-Churn Correlation: {tenure_corr:.3f}")

# Decision
if tenure_corr < -0.2:
    print("\n✓ HYPOTHESIS CONFIRMED: Lower tenure → Higher churn")
    print("  DECISION: Tenure is a strong predictor, use in model")
elif abs(tenure_corr) < 0.1:
    print("\n❌ HYPOTHESIS REJECTED: Tenure shows weak relationship")
    print("  DECISION: Investigate data quality or business context")
else:
    print("\n⚠️ MODERATE RELATIONSHIP: Further investigation needed")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution comparison
axes[0].hist(df[df['Churn']==0]['Tenure'].dropna(), bins=30, alpha=0.6, 
             label='Retained', color='green', edgecolor='black')
axes[0].hist(df[df['Churn']==1]['Tenure'].dropna(), bins=30, alpha=0.6, 
             label='Churned', color='red', edgecolor='black')
axes[0].set_title('Tenure Distribution by Churn Status', fontweight='bold')
axes[0].set_xlabel('Tenure (months)')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
df.boxplot(column='Tenure', by='Churn', ax=axes[1])
axes[1].set_title('Tenure vs Churn (Box Plot)', fontweight='bold')
axes[1].set_xlabel('Churn (0=Retained, 1=Churned)')
axes[1].set_ylabel('Tenure (months)')
plt.suptitle('')
plt.tight_layout()
plt.show()

print("\n📊 INTERPRETATION:")
print("   - Churned customers have LOWER median tenure")
print("   - Confirms early-stage customers are high-risk")
print("   - Feature is SAFE (customer state, not post-churn signal)")

## 😡 Step 7: Complaint Analysis

**Business Hypothesis:** Customers who complained are more likely to churn  
**Expected:** Strong positive correlation

**⚠️ CRITICAL LEAKAGE WARNING:**  
Complaints typically occur **AFTER a bad experience**. This feature may act as a **post-churn signal** rather than a predictive feature.

**Production Reality:**  
- In real systems, complaint may happen DURING the churn process
- Using this feature could give inflated accuracy in training
- May not be available at prediction time (customer hasn't complained yet)

**Decision:**  
- Use feature BUT document leakage risk
- In production, consider lagging (use complaints from previous period)
- Monitor if model relies too heavily on this feature

In [ ]:
print("="*80)
print("COMPLAINT ANALYSIS (WITH LEAKAGE WARNING)")
print("="*80)

# Churn rate by complaint status
complaint_churn = df.groupby('Complain')['Churn'].agg(['count', 'sum', 'mean'])
complaint_churn.columns = ['Total', 'Churned', 'Churn_Rate']
complaint_churn['Churn_Rate'] = (complaint_churn['Churn_Rate'] * 100).round(2)
print("\nChurn Rate by Complaint Status:")
print(complaint_churn)

# Correlation
complain_corr = df['Complain'].corr(df['Churn'])
print(f"\nComplain-Churn Correlation: {complain_corr:.3f}")

# Leakage assessment
print("\n" + "="*80)
print("LEAKAGE RISK ASSESSMENT")
print("="*80)
if complain_corr > 0.3:
    print("⚠️ STRONG POSITIVE CORRELATION DETECTED")
    print("   Complaint likely happens AFTER bad experience")
    print("   DECISION: Use with CAUTION, document leakage risk")
    print("   PRODUCTION STRATEGY: Consider lagging or excluding")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df.groupby(['Complain', 'Churn']).size().unstack().plot(
    kind='bar', ax=axes[0], color=['green', 'red'], edgecolor='black')
axes[0].set_title('Complaint vs Churn (Count)', fontweight='bold')
axes[0].set_xlabel('Complaint Status')
axes[0].set_ylabel('Customer Count')
axes[0].legend(['Retained', 'Churned'])
axes[0].set_xticklabels(['No Complaint', 'Complained'], rotation=0)
axes[0].grid(axis='y', alpha=0.3)

complaint_churn['Churn_Rate'].plot(kind='bar', ax=axes[1], color='orange', edgecolor='black')
axes[1].set_title('Churn Rate by Complaint Status', fontweight='bold')
axes[1].set_xlabel('Complaint Status')
axes[1].set_ylabel('Churn Rate (%)')
axes[1].set_xticklabels(['No Complaint', 'Complained'], rotation=0)
axes[1].axhline(y=df['Churn'].mean()*100, color='red', linestyle='--', 
                label=f'Overall ({df["Churn"].mean()*100:.1f}%)')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 BUSINESS INSIGHT:")
print("   - Customers who complained have SIGNIFICANTLY higher churn")
print("   - This is a strong indicator BUT may be post-churn signal")
print("   - NOT a causal relationship (complaint doesn't CAUSE churn)")
print("   - Complaint is a PROXY for underlying dissatisfaction")

## 🏷️ Step 8: Categorical Features Analysis

**Business Questions:**
- Which payment modes have highest churn?
- Which device types are riskier?
- Does marital status affect loyalty?

**Decision Goal:** Identify high-risk customer segments

In [ ]:
cat_features = ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 
                'PreferedOrderCat', 'MaritalStatus']

print("="*80)
print("CATEGORICAL FEATURES - CHURN RATE ANALYSIS")
print("="*80)

for col in cat_features:
    print(f"\n{col}:")
    churn_by_cat = df.groupby(col)['Churn'].agg(['count', 'sum', 'mean'])
    churn_by_cat.columns = ['Total', 'Churned', 'Churn_Rate']
    churn_by_cat['Churn_Rate'] = (churn_by_cat['Churn_Rate'] * 100).round(2)
    churn_by_cat = churn_by_cat.sort_values('Churn_Rate', ascending=False)
    print(churn_by_cat)
    
    # Identify high-risk categories
    overall_churn = df['Churn'].mean() * 100
    high_risk = churn_by_cat[churn_by_cat['Churn_Rate'] > overall_churn * 1.2]
    if len(high_risk) > 0:
        print(f"  🚨 HIGH-RISK CATEGORIES (>20% above average):")
        for cat in high_risk.index:
            print(f"     - {cat}: {high_risk.loc[cat, 'Churn_Rate']:.1f}%")
    
    print("-" * 80)

## 📊 Step 9: Categorical Visualizations

In [ ]:
fig, axes = plt.subplots(len(cat_features), 1, figsize=(12, 4*len(cat_features)))

for idx, col in enumerate(cat_features):
    churn_rate = df.groupby(col)['Churn'].mean().sort_values(ascending=False) * 100
    churn_rate.plot(kind='bar', ax=axes[idx], color='steelblue', edgecolor='black')
    axes[idx].set_title(f'Churn Rate by {col}', fontweight='bold')
    axes[idx].set_ylabel('Churn Rate (%)')
    axes[idx].set_xlabel(col)
    axes[idx].axhline(y=df['Churn'].mean()*100, color='red', linestyle='--', 
                      label=f'Overall Churn Rate ({df["Churn"].mean()*100:.1f}%)')
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## ✅ Step 10: EDA Key Findings & Modeling Decisions

### 📊 Dataset Summary
- **Total Customers:** 5,630
- **Churn Rate:** ~16.84% (948 churned, 4,682 retained)
- **Class Imbalance:** MODERATE → SMOTE/ADASYN required

---

### 🎯 Strongest Churn Indicators (Correlation Analysis)

**Negative Correlations (↑ feature → ↓ churn):**
- **Tenure:** -0.35 (STRONG) → Older customers more loyal
- **DaySinceLastOrder:** -0.16 → Recent activity reduces churn
- **CashbackAmount:** -0.15 → Higher cashback = retention

**Positive Correlations (↑ feature → ↑ churn):**
- **Complain:** +0.25 (STRONG, ⚠️ LEAKAGE RISK)
- **NumberOfDeviceRegistered:** +0.11
- **SatisfactionScore:** +0.11 (⚠️ LEAKAGE RISK)

---

### 🟢🟡🔴 Feature Trust Assessment

**🟢 SAFE for Production (10 features):**
- Tenure, CityTier, WarehouseToHome
- Demographics: Gender, MaritalStatus
- Preferences: PreferredLoginDevice, PreferredPaymentMode, PreferedOrderCat
- **DECISION:** Use directly in model

**🟡 QUESTIONABLE (4 features):**
- SatisfactionScore, Complain, CashbackAmount, HourSpendOnApp
- **RISK:** May be observed AFTER bad experience
- **DECISION:** Use but DOCUMENT leakage risk, consider lagging

**🔴 HIGH-RISK (4 features):**
- DaySinceLastOrder, OrderCount, OrderAmountHikeFromlastYear, CouponUsed
- **RISK:** Temporal variables, may be post-churn signals
- **DECISION:** Monitor correlation; if > 0.9, likely leakage

---

### 💡 Business Insights Learned

1. **Early-stage customers are high-risk**
   - Lower tenure strongly associated with churn
   - Onboarding/engagement critical in first months

2. **Complaint is a red flag (but may be post-churn)**
   - Customers who complained have 2x+ churn rate
   - NOT causal - complaint is PROXY for dissatisfaction
   - Production: May not be available at prediction time

3. **Payment mode matters**
   - COD has highest churn (~29%)
   - Credit Card has lowest churn (~13%)
   - Hypothesis: Payment convenience affects retention

4. **Device type signals engagement**
   - Phone users churn more than Mobile Phone users
   - May indicate user experience differences

5. **Marital status affects loyalty**
   - Single customers churn more (~27%)
   - Married customers more stable (~12%)

---

### 🚨 Data Quality Issues Identified

- **Missing Values:** Tenure, HourSpendOnApp, Order features
  - **DECISION:** Imputation strategy needed (median for numerical)
  
- **No duplicates, no negative values** → Data quality acceptable

---

### 🚀 Next Steps (TAB 4: Feature Engineering)

**Must Do:**
1. **Handle Missing Values**
   - Median imputation for numerical
   - Mode for categorical

2. **Feature Engineering**
   - RFM features (Recency, Frequency, Monetary)
   - Engagement score (app usage + orders + satisfaction)
   - CLV proxy (tenure × orders × cashback)
   - Complaint rate (complaints / orders)

3. **Handle Class Imbalance**
   - SMOTE for minority class oversampling
   - Stratified train-test split

4. **Feature Selection**
   - Remove high-leakage features if correlation > 0.9
   - Document which features used and why

**Model Strategy:**
- **NOT deep learning** (tabular data, small dataset)
- **Use:** XGBoost, CatBoost, LightGBM
- **Metrics:** ROC-AUC, F1-Score, Precision-Recall (NOT just accuracy)
- **Interpretability:** SHAP values for production trust

---

### ⚠️ Critical Warnings for Modeling

1. **Leakage Risk:** Satisfaction, Complaint features may inflate accuracy
2. **Correlation ≠ Causation:** Don't claim features "cause" churn
3. **Production Reality:** Only use features available at prediction time
4. **Class Imbalance:** Must use SMOTE, not just accuracy metric

---

**EDA COMPLETE. Ready for Feature Engineering (TAB 4).**